In [1]:
import os
import cv2
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, f1_score, r2_score
from tkinter import filedialog
from tkinter import Tk
from math import sqrt
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

In [2]:
# Function to load images from a directory
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            images.append(img)
    return images

In [3]:
# Define the root directory of your dataset
root_dir =  "C:\\Users\\celsi\\Downloads\\archive"

# Define the paths for damaged and intact images
damaged_side_dir = os.path.join(root_dir, 'damaged/side/')
damaged_top_dir = os.path.join(root_dir, 'damaged/top/')
intact_side_dir = os.path.join(root_dir, 'intact/side/')
intact_top_dir = os.path.join(root_dir, 'intact/top/')

# Load images from each directory
damaged_side_images = load_images_from_folder(damaged_side_dir)
damaged_top_images = load_images_from_folder(damaged_top_dir)
intact_side_images = load_images_from_folder(intact_side_dir)
intact_top_images = load_images_from_folder(intact_top_dir)

In [4]:
# Define the dimensions of the images in your dataset
width, height = intact_top_images[0].shape[1], intact_top_images[0].shape[0]




In [5]:
# Create labels for the images
damaged_labels = ['damaged'] * (len(damaged_side_images) + len(damaged_top_images))
intact_labels = ['intact'] * (len(intact_side_images) + len(intact_top_images))



In [6]:
# Concatenate damaged and intact images and labels
all_images = damaged_side_images + damaged_top_images + intact_side_images + intact_top_images
all_labels = damaged_labels + intact_labels



In [7]:
# Convert images to numpy array
all_images = np.array(all_images)



In [8]:
# Flatten the images
all_images_flattened = all_images.reshape(all_images.shape[0], -1)



In [9]:
# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(all_images_flattened, all_labels, test_size=0.1, random_state=42)



In [10]:
# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val) 

# Train a logistic regression model
model = LogisticRegression(random_state=42)  # Increase max_iter if needed
model.fit(X_train, y_train_encoded)


C:\Users\celsi\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [12]:
# Evaluate the model on the validation set
y_val_pred_encoded = model.predict(X_val)
accuracy = accuracy_score(y_val_encoded, y_val_pred_encoded)
print(f"Validation Accuracy for LR algorithm: {accuracy}")

# Evaluate R-squared and RMSE for training set
y_train_pred_encoded = model.predict(X_train)
r2_train = r2_score(y_train_encoded, y_train_pred_encoded)
rmse_train = sqrt(mean_squared_error(y_train_encoded, y_train_pred_encoded))
print(f"Training R-squared: {r2_train}")
print(f"Training RMSE: {rmse_train}")

# Evaluate R-squared and RMSE for testing set
r2_test = r2_score(y_val_encoded, y_val_pred_encoded)
rmse_test = sqrt(mean_squared_error(y_val_encoded, y_val_pred_encoded))
print(f"Testing R-squared: {r2_test}")
print(f"Testing RMSE: {rmse_test}")

# Calculate F1 score
f1 = f1_score(y_val_encoded, y_val_pred_encoded, average='weighted')
print(f"F1 Score: {f1}")


Validation Accuracy for LR algorithm: 0.95
Training R-squared: 1.0
Training RMSE: 0.0
Testing R-squared: 0.7998749218261413
Testing RMSE: 0.22360679774997896
F1 Score: 0.9498114393463231
Confusion Matrix:
[[41  0]
 [ 4 35]]


In [1]:
image_path = "C:\\Users\\celsi\\Downloads\\archive\\damaged\\side\\0753850385083_side.png"

# Load the user-uploaded image
user_image = cv2.imread(image_path)
if user_image is None:
    print("Error loading the image. Exiting.")
else:
    # Resize the image to match the model input dimensions
    resized_image = cv2.resize(user_image, (width, height))

    # Preprocess the image (if needed)
    preprocessed_image = resized_image  # Placeholder for any preprocessing steps

    # Predict the class label
    prediction = model.predict(preprocessed_image.reshape(1, -1))

 # Define colors for bounding boxes
    if prediction[0] == 1:
        color = (0, 255, 0)  # Green for intact
        label = "Intact"
    else:
        color = (0, 0, 255)  # Red for damaged
        label = "Damaged"
    
    # Calculate the bounding box coordinates (assuming the package is in the center of the image)
    box_x1, box_y1 = int(width * 0.25), int(height * 0.25)  # Top-left corner
    box_x2, box_y2 = int(width * 0.75), int(height * 0.75)  # Bottom-right corner

    # Draw a bounding box around the object in the image
    cv2.rectangle(resized_image, (box_x1, box_y1), (box_x2, box_y2), color, thickness=2)
    
    # Add label text
    cv2.putText(resized_image, label, (box_x1, box_y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    
try:
    # Display the image with prediction
    cv2.imshow("Prediction Result", resized_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
except Exception as e:
    print("An error occurred during image display:", e)

NameError: name 'cv2' is not defined